In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize

In [3]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/10_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})
df = df.iloc[:1170]
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m')
df.set_index('Date', inplace=True)
df =df.apply(pd.to_numeric, errors='coerce')
df

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1926-07-01,1.45,15.55,4.69,-1.18,2.90,0.83,0.11,1.77,7.04,2.13
1926-08-01,3.97,3.68,2.81,3.47,2.66,2.17,-0.71,4.25,-1.69,4.35
1926-09-01,1.14,4.80,1.15,-3.39,-0.38,2.41,0.21,0.69,2.04,0.29
1926-10-01,-1.24,-8.23,-3.63,-0.78,-4.58,-0.11,-2.29,-0.57,-2.63,-2.84
1926-11-01,5.20,-0.19,4.10,0.01,4.71,1.63,6.43,5.42,3.71,2.11
...,...,...,...,...,...,...,...,...,...,...
2023-08-01,-3.77,-4.31,-2.37,1.95,-1.68,0.14,-0.40,-0.22,-5.29,-3.35
2023-09-01,-4.57,-2.58,-7.08,3.17,-5.98,-3.22,-5.68,-4.71,-5.04,-3.41
2023-10-01,-3.53,-17.88,-2.76,-6.24,-1.78,-0.18,0.47,-4.58,1.12,-2.53


In [4]:
def rolling_pf_SR(returns, rf, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def negativeSR(w):
            R = np.sum(z_bar * w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            SR = (R - rf) / V
            return -SR

        initial_weights = np.ones(num_assets) / num_assets
        bounds = [(0, None) for _ in range(num_assets)]
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(negativeSR, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()

In [5]:
def rolling_pf_SR_NS(returns, rf, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def negativeSR(w):
            R = np.sum(z_bar * w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            SR = (R - rf) / V
            return -SR

        initial_weights = np.ones(num_assets) / num_assets
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(negativeSR, initial_weights, method='SLSQP', constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()

In [6]:
def rolling_pf_VAR(returns, rf, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def negativeSR(w):
            R = np.sum(z_bar * w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            SR = (R - rf) / V
            return -SR

        initial_weights = np.ones(num_assets) / num_assets
        bounds = [(0, None) for _ in range(num_assets)]
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(negativeSR, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()

In [9]:
def rolling_pf_VOL(returns, window_size):

    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        # Calculer les poids en utilisant la formule
        std = window.std()  # Calcule l'écart-type de chaque actif sur la fenêtre
        weights = (1 / std) / np.sum(1 / std)  # Calculer les poids en utilisant la formule
        
        # Remplir les poids dans le DataFrame
        pf_weights.iloc[i] = weights
        
    return pf_weights.dropna()


rolling_pf_VOL(df, 60)

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1931-07-01,0.147588,0.071878,0.087441,0.093033,0.069723,0.123645,0.095167,0.119455,0.079821,0.112248
1931-08-01,0.147359,0.073626,0.08678,0.093397,0.069311,0.123157,0.095627,0.120043,0.08007,0.110631
1931-09-01,0.147962,0.073623,0.086854,0.092301,0.069302,0.123215,0.095556,0.120344,0.080082,0.110762
1931-10-01,0.139631,0.075963,0.089335,0.094331,0.072203,0.12532,0.09613,0.114042,0.081512,0.111533
1931-11-01,0.139492,0.076184,0.090002,0.093002,0.072552,0.126263,0.094966,0.11309,0.081809,0.112641
...,...,...,...,...,...,...,...,...,...,...
2023-08-01,0.134985,0.044662,0.100351,0.05533,0.094881,0.107341,0.104834,0.131234,0.129311,0.09707
2023-09-01,0.134187,0.044611,0.100208,0.055424,0.095327,0.107603,0.106109,0.131976,0.127737,0.096819
2023-10-01,0.13386,0.045002,0.099666,0.055871,0.095094,0.10823,0.105834,0.13171,0.12739,0.097344


In [10]:
def rolling_pf_VAR(returns, window_size):

    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        # Calculer les poids en utilisant la formule
        var = window.var()  # Calcule l'écart-type de chaque actif sur la fenêtre
        weights = (1 / var) / np.sum(1 / var)  # Calculer les poids en utilisant la formule
        
        # Remplir les poids dans le DataFrame
        pf_weights.iloc[i] = weights
        
    return pf_weights.dropna()


rolling_pf_VAR(df, 60)

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1931-07-01,0.206083,0.04888,0.072338,0.081887,0.045994,0.144641,0.085687,0.135005,0.06028,0.119205
1931-08-01,0.205698,0.05135,0.071337,0.082631,0.045507,0.143679,0.086623,0.136505,0.060732,0.115938
1931-09-01,0.207208,0.051302,0.071397,0.080634,0.045456,0.143691,0.086422,0.137075,0.060698,0.116115
1931-10-01,0.18676,0.055274,0.076448,0.085237,0.049938,0.15044,0.088519,0.12458,0.063644,0.11916
1931-11-01,0.186368,0.05559,0.077585,0.082843,0.050417,0.152696,0.086379,0.122496,0.064102,0.121524
...,...,...,...,...,...,...,...,...,...,...
2023-08-01,0.168357,0.01843,0.093046,0.028287,0.08318,0.106461,0.101547,0.159128,0.1545,0.087063
2023-09-01,0.166503,0.018403,0.092855,0.028406,0.084031,0.107066,0.104113,0.161061,0.150882,0.086681
2023-10-01,0.1659,0.01875,0.091968,0.028902,0.083724,0.108454,0.103704,0.160613,0.150251,0.087734


In [11]:
df.var()

NoDur    20.973675
Durbl    66.290646
Manuf    38.646654
Enrgy    41.118517
HiTec    51.552597
Telcm    21.593690
Shops    33.673078
Hlth     30.489078
Utils    29.903301
Other    40.901173
dtype: float64